In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
%aimport logging

In [33]:
# Set up the logger
def setup_logger(log_file_path):
    """
    Configures the logger to write to a log file and the console.
    """
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        handlers=[
            logging.FileHandler(log_file_path, encoding="utf-8"),
            logging.StreamHandler()  # Optional: to log to the console as well
        ]
    )
    return logging.getLogger(__name__)

In [34]:
# Imports
from data_processing.gpt_processing import (
    set_api_client, 
    generate_messages, 
    create_jsonl_file_for_batch, 
    start_batch, 
    get_batch_response,
    get_completed_batches,
    set_model_settings,
    get_batch_status,
    get_active_batches,
    get_all_batch_info,
    token_count,
    run_immediate_chat_process,
    run_single_batch,
    get_last_batch_response,
    delete_old_files
)

from data_processing.xml_processing import ( 
    save_pages_to_xml,
    split_xml_pages
)

from data_processing.text_processing import (
    get_text_from_file,
    write_text_to_file
)
from pathlib import Path
%aimport time
%aimport json
%aimport datetime

from datetime import datetime

In [35]:
logger = setup_logger("tmp.log")

In [36]:
# Set up API client
client = set_api_client()

2024-11-30 10:10:51,999 - INFO - Open AI API client successfully set: <openai.OpenAI object at 0x11847d7d0>


In [37]:
model_settings = {
    "gpt-4o": {
        "max_tokens": 5000,
        "context_limit": 20000,  # Total context limit for the model
        "temperature": 0.25
    },
    "gpt-3.5-turbo": {
        "max_tokens": 4096,  # Set conservatively to avoid errors
        "context_limit": 16384  # Same as gpt-4o
        }
    }

set_model_settings(model_settings)

In [38]:
# File paths
project_dir = Path("/Users/phapman/Desktop/tnh-scholar/")
data_dir = project_dir / "data_processing"
journal_dir = data_dir / "processed_journal_data"
journal_name = "phat-giao-viet-nam-1956-01"
working_dir = journal_dir / journal_name
input_xml = working_dir / f"TEST2_full_cleaned_{journal_name}.xml"
translated_xml = journal_dir / f"translation_{journal_name}.xml"
section_batch_jsonl = working_dir / "section_batch.jsonl"
translate_batch_jsonl = working_dir / "translation_batch.jsonl"
section_metadata = working_dir / "section_metadata.json"
logfile = data_dir / "gpt_processing" / "processing_info.log"

In [39]:
delete_old_files(datetime.now())

2024-11-30 10:10:54,647 - INFO - HTTP Request: GET https://api.openai.com/v1/files "HTTP/1.1 200 OK"
2024-11-30 10:10:55,039 - INFO - HTTP Request: DELETE https://api.openai.com/v1/files/file-TP94fRELZTYEv7EC65xRby "HTTP/1.1 200 OK"
2024-11-30 10:10:55,041 - INFO - Deleted file: file-TP94fRELZTYEv7EC65xRby (created on 2024-11-30 10:07:58)
2024-11-30 10:10:55,444 - INFO - HTTP Request: DELETE https://api.openai.com/v1/files/file-MNXZLzdmXmLpwsGJdoSuNb "HTTP/1.1 200 OK"
2024-11-30 10:10:55,446 - INFO - Deleted file: file-MNXZLzdmXmLpwsGJdoSuNb (created on 2024-11-30 10:07:41)
2024-11-30 10:10:55,901 - INFO - HTTP Request: DELETE https://api.openai.com/v1/files/file-JcnBXipepk8N6Zf7KwEnk2 "HTTP/1.1 200 OK"
2024-11-30 10:10:55,904 - INFO - Deleted file: file-JcnBXipepk8N6Zf7KwEnk2 (created on 2024-11-30 09:34:57)
2024-11-30 10:10:56,088 - INFO - HTTP Request: DELETE https://api.openai.com/v1/files/file-CYHLaAKtgGkDrkrhQLJLE2 "HTTP/1.1 200 OK"
2024-11-30 10:10:56,092 - INFO - Deleted file: 